# Data Analysis on Chicago Crimes Dataset

First things first, we import all the modules that we are going to use.

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt

%matplotlib inline

Getting the number of lines in each dataset to get a sense of its size.

In [2]:
import mmap

def lines_mapcount(filename):
    f = open(filename, "r+")
    buf = mmap.mmap(f.fileno(), 0)
    lines = 0
    readline = buf.readline
    while readline():
        lines += 1
    return lines

print(f'Chicago Crimes dataset has {lines_mapcount("../data/chicagoCrimes/Chicago_Crimes.json"):,} lines.')
print(f'ZCTA5 dataset has {lines_mapcount("../data/zcta5/TIGER2018_ZCTA5.json"):,} lines.')

Chicago Crimes dataset has 7,147,877 lines.
ZCTA5 dataset has 167 lines.


Loading the Pickle file and converting it into a GeoPandas DataFrame

In [3]:
file_name = "../data/chicagoCrimes/Chicago_Crimes_cleaned.pkl"
dfCrime = pd.read_pickle(file_name)
dfCrime = gpd.GeoDataFrame(dfCrime, geometry='g')
print("Loading done!")

Loading done!


Here we take a look on the first 5 rows of the dataset.

In [4]:
dfCrime.head()

,g,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,District,FBI Code,Score Crime,Score Arrest
0,POINT (-87.67920 41.96925),9799787,2014-09-30 01:05:00,CRIMINAL TRESPASS,TO VEHICLE,STREET,1,0,20.0,26,1,<NA>
1,POINT (-87.67920 41.96930),9933145,2015-01-20 15:30:00,CRIMINAL DAMAGE,TO PROPERTY,SMALL RETAIL STORE,0,0,20.0,14,13,<NA>
2,POINT (-87.67772 41.96947),9936081,2015-01-21 11:00:00,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,0,0,20.0,14,13,<NA>
3,POINT (-87.67772 41.96947),9833072,2014-10-25 12:00:00,CRIMINAL DAMAGE,TO VEHICLE,APARTMENT,0,0,20.0,14,13,<NA>
4,POINT (-87.67772 41.96949),9822109,2014-10-17 08:30:00,THEFT,$500 AND UNDER,RESIDENTIAL YARD (FRONT/BACK),0,0,20.0,06,21,<NA>


Then we get general info of each column; like, the datatype, abd the non-null values count.

In [5]:
dfCrime.info(memory_usage="deep")

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7078918 entries, 0 to 7147876
Data columns (total 12 columns):
 #   Column                Dtype         
---  ------                -----         
 0   g                     geometry      
 1   ID                    Int32         
 2   Date                  datetime64[ns]
 3   Primary Type          category      
 4   Description           category      
 5   Location Description  category      
 6   Arrest                Int8          
 7   Domestic              Int8          
 8   District              category      
 9   FBI Code              category      
 10  Score Crime           Int8          
 11  Score Arrest          Int8          
dtypes: Int32(1), Int8(4), category(5), datetime64[ns](1), geometry(1)
memory usage: 297.1 MB


Then a quick descriptive statistics summary of numeric columns.

In [6]:
dfCrime.describe()

,ID,Arrest,Domestic,Score Crime,Score Arrest
count,7.078918e+06,7.078918e+06,7.078918e+06,7.078918e+06,0
mean,6.531848e+06,2.735994e-01,1.339089e-01,1.604244e+01,<NA>
std,3.214072e+06,4.458058e-01,3.405544e-01,6.778636e+00,<NA>
min,6.340000e+02,0.000000e+00,0.000000e+00,1.000000e+00,<NA>
25%,3.561036e+06,0.000000e+00,0.000000e+00,1.300000e+01,<NA>
50%,6.521622e+06,0.000000e+00,0.000000e+00,1.900000e+01,<NA>
75%,9.310905e+06,1.000000e+00,0.000000e+00,2.100000e+01,<NA>
max,1.209505e+07,1.000000e+00,1.000000e+00,2.600000e+01,<NA>


Then a quick overview of the number of unique values in each column.

In [7]:
dfCrime.nunique()

g                        872127
ID                      7078918
Date                    2882091
Primary Type                 36
Description                 522
Location Description        212
Arrest                        2
Domestic                      2
District                     24
FBI Code                     26
Score Crime                  23
Score Arrest                  0
dtype: int64

In [8]:
dfZipCode = pd.read_json("../data/zcta5/TIGER2018_ZCTA5.json", lines=True)

In [9]:
dfZipCode.drop(columns=["ZCTA5CE10", "CLASSFP10", "MTFCC10", "FUNCSTAT10"], inplace=True)
dfZipCode[["GEOID10", "ALAND10", "AWATER10"]] = dfZipCode[["GEOID10", "ALAND10", "AWATER10"]].astype('Int32')
dfZipCode['g'] = gpd.GeoSeries.from_wkt(dfZipCode['g'])
dfZipCode = gpd.GeoDataFrame(dfZipCode, geometry='g')

In [10]:
dfZipCode.head()

,g,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
0,"POLYGON ((-88.20610 41.74995, -88.20606 41.752...",60540,34212403,544116,41.764670,-88.145428
1,"POLYGON ((-88.17978 41.83637, -88.17970 41.836...",60189,26452325,885699,41.840632,-88.122024
2,"POLYGON ((-88.16752 41.72579, -88.16747 41.726...",60565,32161307,482552,41.730729,-88.124336
3,"POLYGON ((-88.16364 41.62359, -88.16351 41.623...",60446,43301976,2034401,41.631698,-88.106903
4,"MULTIPOLYGON (((-88.08942 41.61425, -88.08932 ...",60441,72613199,1531111,41.592639,-88.050043


In [11]:
dfZipCode.info(memory_usage="deep")

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   g           167 non-null    geometry
 1   GEOID10     167 non-null    Int32   
 2   ALAND10     167 non-null    Int32   
 3   AWATER10    167 non-null    Int32   
 4   INTPTLAT10  167 non-null    float64 
 5   INTPTLON10  167 non-null    float64 
dtypes: Int32(3), float64(2), geometry(1)
memory usage: 6.5 KB


In [12]:
dfZipCode.describe()

,GEOID10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
count,167.000000,1.670000e+02,1.670000e+02,167.000000,167.000000
mean,59771.431138,1.627824e+07,5.952942e+05,41.838319,-87.787317
std,3026.114349,1.633139e+07,1.302942e+06,0.130454,0.180256
min,46304.000000,2.254770e+05,0.000000e+00,41.590820,-88.145428
25%,60172.500000,7.000086e+06,0.000000e+00,41.741223,-87.900872
50%,60491.000000,1.227981e+07,1.134700e+05,41.857903,-87.788946
75%,60622.500000,1.986572e+07,5.867145e+05,41.939528,-87.665181
max,60827.000000,1.388441e+08,9.157099e+06,42.069786,-87.046205


In [13]:
dfZipCode.nunique()

g             167
GEOID10       167
ALAND10       167
AWATER10      115
INTPTLAT10    167
INTPTLON10    167
dtype: int64

---

1. Which police district has the most crimes?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT District, COUNT(1) AS NumberOfCrimes
    FROM ChicagoCrimes
    GROUP BY District
    ORDER BY NumberOfCrimes DESC
    LIMIT 1;
    ```

    Pandas:

In [14]:
dfCrime.groupby("District", as_index=False).size().sort_values(by=['size'], ascending=False)[['District', 'size']].iloc[0]

District         8.0
ID          479853.0
Name: 7, dtype: float64

2. Which police district has the most arrest percentage?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT District, SUM(int32(boolean(Arrest)))/COUNT(1) AS PercentageOfArrests
    FROM ChicagoCrimes
    GROUP BY District
    ORDER BY PercentageOfArrests DESC
    LIMIT 1;
    ```

    Pandas:


In [15]:
district_group = dfCrime.groupby('District', as_index=True)
(district_group['Arrest'].sum() / district_group['Arrest'].count()).sort_values(ascending=False).iloc[[0]]

District
21.0    0.5
Name: Arrest, dtype: Float64

3. How many murder crimes happen on the street?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT COUNT(1) AS NumberOfCrimesOnStreet
    FROM ChicagoCrimes
    WHERE `Location Description` = "STREET" AND `Primary Type` = "HOMICIDE";
    ```

    Pandas:


In [16]:
dfCrime[(dfCrime['Primary Type'] == 'HOMICIDE') & (dfCrime['Location Description'] == 'STREET')].count()['ID']

5118

4. Where does murder crimes usually happen? (ZIP Code)

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT z.GEOID10, COUNT(1) AS NumberOfCrimes
    FROM ChicagoCrimes c JOIN ZipCodes z ON st_contains(z.g, c.g)
    GROUP BY z.`GEOID10`
    ORDER BY NumberOfCrimes DESC
    LIMIT 5;
    ```

    Pandas:

In [17]:
tmp = dfCrime.sjoin(dfZipCode, how='inner')
tmp.groupby('GEOID10', as_index=False).size().sort_values(by='size', ascending=False).reset_index(drop=True).iloc[:5]

,GEOID10,size
0,60628,307647
1,60620,303795
2,60624,269733
3,60644,267700
4,60619,265053


5. Where does murder crimes usually happen? (Location Description)

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT `Location Description`, COUNT(1) AS NumberOfCrimes
    FROM ChicagoCrimes
    WHERE `Primary Type` = "HOMICIDE"
    GROUP BY `Location Description`
    ORDER BY `NumberOfCrimes` DESC;
    LIMIT 5;
    ```

    Pandas:

In [19]:
tmp_df = dfCrime[(dfCrime['Primary Type'] == 'HOMICIDE')].groupby('Location Description', as_index=False).size().sort_values(by=['size'], ascending=False).iloc[0:5]
print(tmp_df)

    Location Description  size
186               STREET  5118
21                  AUTO  1193
17             APARTMENT   886
15                 ALLEY   660
115                HOUSE   552


6. What is the time of day when murder and kidnapping happen the most?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT CrimeHour, COUNT(1) AS NumberOfCrimes
    FROM ChicagoCrimes
    WHERE `Primary Type` = "HOMICIDE" OR `Primary Type` = "KIDNAPPING"
    GROUP BY get_hour(parse_datetime(Date,"M/D/Y h:m:s a")) AS CrimeHour
    ORDER BY NumberOfCrimes DESC
    LIMIT 1;
    ```

    Pandas:


In [18]:
tmp_df = dfCrime[(dfCrime['Primary Type'] == 'HOMICIDE') | (dfCrime['Primary Type'] == 'KIDNAPPING')].groupby(dfCrime['Date'].dt.hour, as_index=False).size().sort_values(by=['size'], ascending=False).iloc[:1]
print(tmp_df)

    Date  size
18    18  1024


7. What is the most common domestic crime?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT `Primary Type`, COUNT(1) AS NumberOfCrimes
    FROM ChicagoCrimes
    WHERE `Domestic` = "true"
    GROUP BY `Primary Type`
    ORDER BY NumberOfCrimes DESC
    LIMIT 1;
    ```

    Pandas:

In [20]:
dfCrime[dfCrime['Domestic'] == 1].groupby('Primary Type', as_index=False).size().sort_values('size').iloc[-1]

Primary Type    BATTERY
size             559295
Name: 2, dtype: object

8. What is the percentage of domestic crimes that led to an arrest?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT SUM(int32(boolean(Arrest)))/COUNT(1) AS PercentageOfArrests
    FROM ChicagoCrimes
    WHERE Domestic='true';
    ```

    Pandas:

In [21]:
tmp_df = dfCrime[dfCrime['Domestic'] == 1]
tmp_df['Arrest'].sum() / tmp_df['Arrest'].size

0.19576129039064066

9. Which day of week has the most domestic crimes?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT Day, COUNT(1) AS NumberOfCrimes
    FROM ChicagoCrimes
    WHERE`Domestic` = 'true'
    GROUP BY day_of_week(parse_datetime(Date,"M/D/Y h:m:s a")) AS Day
    ORDER BY NumberOfCrimes DESC
    LIMIT 1;
    ```

    Pandas:

In [22]:
dfCrime[dfCrime['Domestic'] == 1].groupby(dfCrime['Date'].dt.dayofweek).size().sort_values().iloc[-1]

160171

10. What is the most common crime in each day of week?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    WITH CrimesPerDay AS (
      SELECT Day, `Primary Type`, COUNT(1) AS NumberOfCrimes
      FROM ChicagoCrimes c
      GROUP BY day_of_week(parse_datetime(Date,"M/D/Y h:m:s a")) AS Day, `Primary Type`
    )

    SELECT a.Day, a.`Primary Type`, a.NumberOfCrimes
    FROM CrimesPerDay a INNER JOIN (
        SELECT Day, MAX(NumberOfCrimes) AS NumberOfCrimes
        FROM CrimesPerDay
        GROUP BY Day
    ) b ON a.Day = b.Day AND a.NumberOfCrimes = b.NumberOfCrimes
    ORDER BY a.Day;
    ```

    Pandas:

In [23]:
dfCrime['dayofweek'] = (dfCrime['Date'].dt.dayofweek+1)%7 # get day of week and make it start from sunday
tmp = dfCrime.groupby(by=['dayofweek', 'Primary Type'], as_index=False).size()
tmp[tmp.groupby('dayofweek')['size'].rank('dense', ascending=False) == 1]

,dayofweek,Primary Type,size
2,0,BATTERY,214738
70,1,THEFT,216121
106,2,THEFT,216160
142,3,THEFT,217400
178,4,THEFT,214626
214,5,THEFT,231048
250,6,THEFT,211503


11. Which month generally has the greatest number of crimes?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT Month, COUNT(1) AS NumberOfCrimes
    FROM ChicagoCrimes
    GROUP BY get_month(parse_datetime(Date,"M/D/Y h:m:s a")) AS Month
    ORDER BY NumberOfCrimes DESC
    LIMIT 1;
    ```

    Pandas:


In [24]:
dfCrime['month'] = dfCrime['Date'].dt.month
dfCrime.groupby('month', as_index=False).size().sort_values('size').iloc[-1]

month         7
size     650441
Name: 6, dtype: int64

12. What is the time of day when theft-related crimes happen the most?

    AsterixDB:
    ```SQL
    USE GeospatialAnalysis;

    SELECT CrimeHour, COUNT(1) AS NumberOfCrimes
    FROM ChicagoCrimes
    WHERE `Primary Type` = "MOTOR VEHICLE THEFT" OR
        `Primary Type` = "BURGLARY" OR
        `Primary Type` = "ROBBERY" OR
        `Primary Type` = "THEFT"
    GROUP BY get_hour(parse_datetime(Date,"M/D/Y h:m:s a")) AS CrimeHour
    ORDER BY NumberOfCrimes DESC
    LIMIT 1;
    ```

    Pandas:

In [25]:
dfCrime['hour'] = dfCrime['Date'].dt.hour
theftRelatedList = ["MOTOR VEHICLE THEFT", "BURGLARY", "ROBBERY", "THEFT"]
dfCrime[
    (dfCrime['Primary Type'] == "MOTOR VEHICLE THEFT") |
    (dfCrime['Primary Type'] == "BURGLARY") |
    (dfCrime['Primary Type'] == "ROBBERY") |
    (dfCrime['Primary Type'] == "THEFT")
    ].groupby('hour', as_index=False).size().sort_values('size').iloc[-1]

hour        12
size    151309
Name: 12, dtype: int64

13. What is the most common crime in each area?

    AsterixDB
    ```SQL
    USE GeospatialAnalysis;

    WITH CrimesPerZipCode AS (
    SELECT z.GEOID10 AS `Zip Code`, c.`Primary Type`, COUNT(1) AS NumberOfCrimes
    FROM ChicagoCrimes c JOIN ZipCodes z ON st_contains(z.g, c.g) = true
    GROUP BY z.GEOID10, c.`Primary Type`
    )

    SELECT a.`Zip Code`, a.`Primary Type`, a.NumberOfCrimes
    FROM CrimesPerZipCode a INNER JOIN (
        SELECT `Zip Code`, MAX(NumberOfCrimes) AS NumberOfCrimes
        FROM CrimesPerZipCode
        GROUP BY `Zip Code`
    ) b ON a.`Zip Code` = b.`Zip Code` AND a.NumberOfCrimes = b.NumberOfCrimes
    ```

    Pandas:

In [26]:
joint_df = dfCrime.sjoin(dfZipCode, how='inner')
tmp = joint_df.groupby(by=['GEOID10', 'Primary Type'], as_index=False).size()
tmp[tmp.groupby('GEOID10')['size'].rank('first', ascending=False) == 1]

,GEOID10,Primary Type,size
6,46320,CRIMINAL DAMAGE,9
37,60007,ASSAULT,1
106,60018,THEFT,8640
142,60068,THEFT,447
146,60076,BATTERY,4
...,...,...,...
2842,60714,THEFT,79
2846,60803,BATTERY,88
2914,60804,THEFT,202
2950,60805,THEFT,733
